In [4]:
import nidaqmx.system
system = nidaqmx.system.System.local()
system.driver_version

DriverVersion(major_version=21, minor_version=0, update_version=0)

In [1]:
from ..DAQcontrol import *


ImportError: attempted relative import with no known parent package

In [6]:
import os; os.getcwd()

'd:\\Brateen\\Python_codes\\qdSpectro-active\\NV_Experiment\\python_examples_trial_codes'

In [2]:
for device in system.devices:
    print(device)

Device(name=P6363)
Device(name=U9263)


Basic AO

In [2]:
import numpy as np, matplotlib.pyplot as plt, nidaqmx, nidaqmx.constants, time, scipy as sp
from scipy import signal
%matplotlib qt5

rate = 1000; dt = 1/rate; print(dt)
n_points = int((1-0)/dt); print(n_points)
t = np.linspace(0,1,n_points,endpoint=False)
f=2
# create the dc+square_wave pattern
# data = np.zeros(np.shape(t))
data = np.cos(2*np.pi*f*t)
# data = np.append(data,data)
# data=[]
# data = np.append(data,signal.square(2*np.pi*f*t, duty=0.5))
# data = np.append(data,np.zeros(np.shape(t)))
# data = np.append(data,signal.square(2*np.pi*f*t, duty=0.5))
plt.plot(np.array(range(0,len(data)))*dt,data,'.');
plt.xlabel('Time [s]'); #plt.title('cos(2*np.pi*f*t), f=%d, 1e5 points' %f);

0.001
1000


In [4]:
len(data)

1000

In [1]:
Nsamples = len(data)*5
timeout = Nsamples/rate; print("Runtime / Timeout [s]:", timeout)
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("U9263/ao2")
    task.timing.cfg_samp_clk_timing(rate, source='', active_edge=nidaqmx.constants.Edge.RISING, sample_mode=nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=Nsamples)
    task.write(data, auto_start=True)
    time.sleep(timeout)
    # task.write(0)
    # task.close()
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("U9263/ao2")
    task.timing.cfg_samp_clk_timing(rate, source='', active_edge=nidaqmx.constants.Edge.RISING, sample_mode=nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=10)
    task.write([0]*10, auto_start=True)


NameError: name 'data' is not defined

Advanced AO: Different Patterns

In [9]:
import numpy as np, matplotlib.pyplot as plt, nidaqmx, nidaqmx.constants, time, scipy as sp
from scipy import signal
%matplotlib qt5

coil_calib = [26.5, 26.8, 43.5]     # in gauss for 1 V_DAQ volatge; specified for (x,y,z) coils respectively..
offset = 25; amplitude_z = [30];      # offset and amplitude_z of magetic field in gauss
offset = offset/coil_calib[2]; amplitude_z = [i/coil_calib[2] for i in amplitude_z]
rate = 100; dt = 1/rate; print('dt =',dt)      # dt represents the time difference between the two consecutive points in the time arrray

# create the dc+square_wave pattern
pattern_time = 120;  # pattern time in seconds
n_points = int((pattern_time-0)/dt); print('n_points =', n_points)
t = np.linspace(0, pattern_time, n_points, endpoint=False)
# The above statement is equivalent to MATLAB t = np.arange(0,pattern_time,dt), the last time point is ignored..
# The statements produce the required time array with the spacing as specified by the SAMPLE RATE... Remember that the sample rate is the deciding factor...
data = offset + np.zeros(np.shape(t));

def create_square_test_field(t, offset, amp, freq):
    # n_patterns = 2; pattern = np.zeros((len(t),n_patterns))
    # # for i in range(0,n_patterns):
    # pattern[:,0] = 
    pattern = offset + (amp-offset) * signal.square(2*np.pi*freq*t, duty=0.5)
    # print(pattern)
    # pattern = np.ravel(pattern,order='F')
    # print(pattern)
    return pattern

frequency=1/60;
# data=[]
pattern_time = 900;  # pattern time in seconds
n_points = int((pattern_time-0)/dt); print('n_points =', n_points)
t = np.linspace(0, pattern_time, n_points, endpoint=False)
for i in range(0, len(amplitude_z)):
    data = np.append(data, create_square_test_field(t, offset, amp=amplitude_z[i], freq=frequency))

print('len(data) =', len(data))
plt.subplot(211);   plt.plot(np.array(range(0, len(data)))*dt, data, '--'); plt.ylabel("DAQ Voltage [V]");
plt.subplot(212);   plt.plot(np.array(range(0, len(data)))*dt, data*coil_calib[2], '--'); plt.ylabel("Magnetic field [G]");
plt.xlabel('Time [s]'); #plt.title('cos(2*np.pi*f*t), f=%d, 1e5 points' %f);

dt = 0.01
n_points = 12000
n_points = 90000
len(data) = 102000


In [10]:
plt.figure(); plt.plot(np.array(range(0, len(data)))*dt, data*coil_calib[2], '--'); plt.ylabel("Magnetic field [G]");
plt.xlabel('Time [s]')

Text(0.5, 0, 'Time [s]')

In [2]:
Nsamples = len(data)
timeout = Nsamples/rate; print("Runtime / Timeout [s]:", timeout)
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("U9263/ao2")
    task.timing.cfg_samp_clk_timing(rate, source='', active_edge=nidaqmx.constants.Edge.RISING, sample_mode=nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=Nsamples)
    task.write(data, auto_start=True)
    time.sleep(timeout)
    # task.write(0)
    # task.close()
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("U9263/ao2")
    task.timing.cfg_samp_clk_timing(rate, source='', active_edge=nidaqmx.constants.Edge.RISING, sample_mode=nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=10)
    task.write([0]*10, auto_start=True)


Runtime / Timeout [s]: 1020.0


In [1]:
import numpy as np, matplotlib.pyplot as plt, nidaqmx, nidaqmx.constants, time, scipy as sp
from scipy import signal
%matplotlib qt5

# coil_calib = [26.5, 26.8, 43.5]     # in gauss for 1 V_DAQ volatge; specified for (x,y,z) coils respectively..
offset = 0; amplitude_z = [5];      # offset and amplitude_z of magetic field in gauss
# offset = offset/coil_calib[2]; amplitude_z = [i/coil_calib[2] for i in amplitude_z]
rate = 200e2; dt = 1/rate; print('dt =',dt)      # dt represents the time difference between the two consecutive points in the time arrray

# create the sine_wave pattern

def create_square_test_field(t, offset, amp, freq):
    # n_patterns = 2; pattern = np.zeros((len(t),n_patterns))
    # # for i in range(0,n_patterns):
    # pattern[:,0] = 
    pattern = offset + (amp-offset) * signal.square(2*np.pi*freq*t, duty=0.5)
    # print(pattern)
    # pattern = np.ravel(pattern,order='F')
    # print(pattern)
    return pattern

def create_sine_field(t, offset, amp, freq):
    pattern = offset + (amp-offset) * np.sin(2*np.pi*freq*t)
    return pattern

frequency=117;
data=[]
pattern_time = 10;  # pattern time in seconds
n_points = int((pattern_time-0)/dt); print('n_points =', n_points)
t = np.linspace(0, pattern_time, n_points, endpoint=False)
for i in range(0, len(amplitude_z)):
    data = np.append(data, create_sine_field(t, offset, amp=amplitude_z[i], freq=frequency))

print('len(data) =', len(data))

plt.figure(); plt.plot(np.array(range(0, len(data)))*dt, data); plt.ylabel("DAQ Voltage [V]");
# plt.subplot(212);   plt.plot(np.array(range(0, len(data)))*dt, data*coil_calib[2], '--'); plt.ylabel("Magnetic field [G]");
plt.xlabel('Time [s]'); #plt.title('cos(2*np.pi*f*t), f=%d, 1e5 points' %f);

dt = 5e-05
n_points = 200000
len(data) = 200000


In [3]:
ao_device = "U6211/ao0"
Nsamples = len(data)
timeout = Nsamples/rate; print("Runtime / Timeout [s]:", timeout)
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan(ao_device)
    task.timing.cfg_samp_clk_timing(rate, source='OnboardClock', active_edge=nidaqmx.constants.Edge.RISING, sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS, samps_per_chan=Nsamples)
    task.write(data, auto_start=False)
    def callback(task_handle, every_n_samples_event_type, number_of_samples, callback_data):
        task.write(data, auto_start=False)
        return 0
    
    task.register_every_n_samples_transferred_from_buffer_event(int(Nsamples*0.1), callback)
    task.start()
    # task.register_every_n_samples_transferred_from_buffer_event

    # print("Primes upto 1e5:")
    # start_time = time.perf_counter();
    # for i in range(0,int(1e5)):
    #     c=0;
    #     for j in range(1,int(i)):
    #         if i%j==0:
    #             c+=1
    #     if c==1:
    #         print(i)
    # stop_time = time.perf_counter();
    input("Press enter to stop")
    # task.write(0)
    # task.close()

# print("Loop time [min] = ", (stop_time-start_time)/60)
# with nidaqmx.Task() as task:
#     task.ao_channels.add_ao_voltage_chan("U6211/ao0")
#     task.timing.cfg_samp_clk_timing(rate, source='', active_edge=nidaqmx.constants.Edge.RISING, sample_mode=nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=10)
#     task.write([0]*10, auto_start=True)

Runtime / Timeout [s]: 10.0


what is error code: -200877??

Background acquisition example

In [4]:
"""Example for reading singals for every n samples."""
import pprint, matplotlib.pyplot as plt
%matplotlib qt5
import nidaqmx, time
from nidaqmx.constants import AcquisitionType, TerminalConfiguration

pp = pprint.PrettyPrinter(indent=4)


with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("P6363/ai8", terminal_config=TerminalConfiguration.RSE, )

    task.timing.cfg_samp_clk_timing(1000, sample_mode=AcquisitionType.CONTINUOUS)

    samples = []
    # plt.figure(1)
    def callback(task_handle, every_n_samples_event_type, number_of_samples, callback_data):
        """Callback function for reading singals."""
        # print("Every N Samples callback invoked.")
        samp = task.read(number_of_samples_per_channel=1000)
        samples.extend(samp)
        # plt.plot(samp)
        time.sleep(2)
        # plt.show()
        return 0

    task.register_every_n_samples_acquired_into_buffer_event(1000, callback)

    task.start()
    for i in range(0,10):
        print(i)
    input("Running task. Press Enter to stop and see number of " "accumulated samples.\n")

plt.plot(samples)
print(len(samples))

0
1
2
3
4
5
6
7
8
9
6000


Background generation trial

'Real-time' plotting from Halsvorsen blog

In [ ]:
# https://www.halvorsen.blog/documents/programming/python/resources/powerpoints/DAQ%20with%20Python.pdf
import nidaqmx
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# Read from DAQ Device
def readdaq():
    task = nidaqmx.Task()
    task.ai_channels.add_ai_voltage_chan("P6363/ai8", terminal_config=nidaqmx.constants.TerminalConfiguration.RSE)
    task.start()
    value = task.read()
    task.stop()
    task.close()
    return value

# Write Data Function
def writefiledata(t, x):
    # Open File
    file = open("tempdata.txt", "a")
    # Write Data
    time = str(t)
    value = str(round(x, 2))
    file.write(time + "\t" + value)
    file.write("\n")
    # Close File
    file.close()

# Initialize Logging
Ts = 1 # Sampling Time [seconds]
N = 100
k = 1
x_len = N # Number of points to display
Tmin = 15; Tmax = 28
# y_range = [Tmin, Tmax] # Range of possible Y values to display
data = []

# Create figure for plotting
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
xs = list(range(0, N))
ys = [0] * x_len
# ax.set_ylim(y_range)

# Create a blank line. We will update the line in animate
line, = ax.plot(xs, ys)
# Configure Plot
plt.title('Temperature')
plt.xlabel('t [s]')
plt.ylabel('Temp [degC]')
plt.grid()

#Logging Temperature Data from DAQ Device
def logging(i, ys):
    value = readdaq()
    print("T =", round(value,1), "[degC]")
    data.append(value)
    time.sleep(Ts)
    global k
    k = k + 1
    writefiledata(k*Ts, value)
    # Add y to list
    ys.append(value)
    # Limit y list to set number of items
    ys = ys[-x_len:]
    # Update line with new Y values
    line.set_ydata(ys)
    return line,
ani = animation.FuncAnimation(fig, logging, fargs=(ys,), interval=100, blit=True)
plt.show()

In [7]:
"""Simultaneous read and write with NI USB-4431 or similar device."""

import numpy as np

import nidaqmx as ni
from nidaqmx.constants import WAIT_INFINITELY


def query_devices():
    """Queries all the device information connected to the local system."""
    local = ni.system.System.local()
    for device in local.devices:
        print(f"Device Name: {device.name}, Product Type: {device.product_type}")
        print("Input channels:", [chan.name for chan in device.ai_physical_chans])
        print("Output channels:", [chan.name for chan in device.ao_physical_chans])


def playrec(data, samplerate, input_mapping, output_mapping):
    """Simultaneous playback and recording though NI device.

    Parameters:
    -----------
    data: array_like, shape (nsamples, len(output_mapping))
      Data to be send to output channels.
    samplerate: int
      Samplerate
    input_mapping: list of str
      Input device channels
    output_mapping: list of str
      Output device channels

    Returns
    -------
    ndarray, shape (nsamples, len(input_mapping))
      Recorded data

    """
    devices = ni.system.System.local().devices
    data = np.asarray(data).T
    nsamples = data.shape[1]

    with ni.Task() as read_task, ni.Task() as write_task:
        for i, o in enumerate(output_mapping):
            aochan = write_task.ao_channels.add_ao_voltage_chan(
                o,
                min_val=devices[o].ao_voltage_rngs[0],
                max_val=devices[o].ao_voltage_rngs[1],
            )
            min_data, max_data = np.min(data[i]), np.max(data[i])
            if ((max_data > aochan.ao_max) | (min_data < aochan.ao_min)).any():
                raise ValueError(
                    f"Data range ({min_data:.2f}, {max_data:.2f}) exceeds output range of "
                    f"{o} ({aochan.ao_min:.2f}, {aochan.ao_max:.2f})."
                )
        for i in input_mapping:
            read_task.ai_channels.add_ai_voltage_chan(i)

        for task in (read_task, write_task):
            task.timing.cfg_samp_clk_timing(
                rate=samplerate, source="OnboardClock", samps_per_chan=nsamples
            )

        # trigger write_task as soon as read_task starts
        write_task.triggers.start_trigger.cfg_dig_edge_start_trig(
            read_task.triggers.start_trigger.term
        )
        # squeeze as Task.write expects 1d array for 1 channel
        write_task.write(data.squeeze(), auto_start=False)
        # write_task doesn't start at read_task's start_trigger without this
        write_task.start()
        # do not time out for long inputs
        indata = read_task.read(nsamples, timeout=WAIT_INFINITELY)

    return np.asarray(indata).T


if __name__ == "__main__":
    query_devices()
    # Prints in this example:
    #   Device Name: Dev2, Product Type: USB-4431
    #   Input channels: ['Dev2/ai0', 'Dev2/ai1', 'Dev2/ai2', 'Dev2/ai3']
    #   Output channels: ['Dev2/ao0']

    # excite through one output and record at three inputs
    outdata = np.random.normal(size=(5000, 1)) * 0.01
    indata = playrec(
        outdata,
        samplerate=96000,
        input_mapping=["P6363/ai0", "P6363/ai1", "P6363/ai2"],
        output_mapping=["P6363/ao0"],
    )


Device Name: P6363, Product Type: PCIe-6363
Input channels: ['P6363/ai0', 'P6363/ai1', 'P6363/ai2', 'P6363/ai3', 'P6363/ai4', 'P6363/ai5', 'P6363/ai6', 'P6363/ai7', 'P6363/ai8', 'P6363/ai9', 'P6363/ai10', 'P6363/ai11', 'P6363/ai12', 'P6363/ai13', 'P6363/ai14', 'P6363/ai15', 'P6363/ai16', 'P6363/ai17', 'P6363/ai18', 'P6363/ai19', 'P6363/ai20', 'P6363/ai21', 'P6363/ai22', 'P6363/ai23', 'P6363/ai24', 'P6363/ai25', 'P6363/ai26', 'P6363/ai27', 'P6363/ai28', 'P6363/ai29', 'P6363/ai30', 'P6363/ai31']
Output channels: ['P6363/ao0', 'P6363/ao1', 'P6363/ao2', 'P6363/ao3']
